# Multi-table Datasets - ENRON Archive

## 1. Data import

Connect to the file 'assets/datasets/enron.db' using one of these methods:

- sqlite3 python package
- pandas.read_sql
- SQLite Manager Firefox extension

Take a look at the database and query the master table. How many Tables are there in the db?

> Answer:
There are 3 tables:
- MessageBase
- RecipientBase
- EmployeeBase

In [6]:
import sqlite3
import pandas
sqlite_db = '../../assets/datasets/enron.db'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()

In [8]:
sql = """
SELECT * from sqlite_master
WHERE type = 'table' """

df = pd.read_sql(sql, con=conn)
df

,type,name,tbl_name,rootpage,sql
0,table,MessageBase,MessageBase,3,"CREATE TABLE MessageBase (\n mid INTEGER,\n..."
1,table,RecipientBase,RecipientBase,5,"CREATE TABLE RecipientBase (\n mid INTEGER,..."
2,table,EmployeeBase,EmployeeBase,2,CREATE TABLE EmployeeBase (\n ...


In [ ]:
# import pandas as pd
# data = pd.read_csv('../../assets/datasets/enron.db')
# data.to_sql('enron',
#            con = conn,
#            if_exists = 'replace', 
#            index = False)

Query the `sqlite_master` table to retrieve the schema of the `EmployeeBase` table.

1. What fields are there?
1. What's the type of each of them?

In [27]:
empbase = """
SELECT * 
FROM EmployeeBase"""

employee_base = pd.read_sql(empbase, con=conn)

In [21]:
employee_base_fields = list(employee_base.columns)
employee_base_fields

['eid', 'name', 'department', 'longdepartment', 'title', 'gender', 'seniority']

In [22]:
employee_base.dtypes

eid                int64
name              object
department        object
longdepartment    object
title             object
gender            object
seniority         object
dtype: object

1. Print the first 5 rows of EmployeeBase table
1. Print the first 5 rows of MessageBase table
1. Print the first 5 rows of RecipientBase table

**Hint**  use `SELECT` and `LIMIT`.

Import each of the 3 tables to a Pandas Dataframes

In [28]:
employee_base.head()

,eid,name,department,longdepartment,title,gender,seniority
0,1,John Arnold,Forestry,ENA Gas Financial,VP Trading,Male,Senior
1,2,Harry Arora,Forestry,ENA East Power,VP Trading,Male,Senior
2,3,Robert Badeer,Forestry,ENA West Power,Mgr Trading,Male,Junior
3,4,Susan Bailey,Legal,ENA Legal,Specialist Legal,Female,Junior
4,5,Eric Bass,Forestry,ENA Gas Texas,Trader,Male,Junior


In [45]:
msgbase = """
SELECT *
FROM MessageBase"""

message_base = pd.read_sql(msgbase, con=conn)
message_base.head()

,mid,filename,unix_time,subject,from_eid
0,1,taylor-m/sent/11,910930020,Cd$ CME letter,138
1,2,taylor-m/sent/17,911459940,Indemnification,138
2,3,taylor-m/sent/18,911463840,Re: Indemnification,138
3,4,taylor-m/sent/23,911874180,"Re: Coral Energy, L.P.",138
4,5,taylor-m/sent/27,912396120,Bankruptcy Code revisions,138


In [46]:
recipient = """
SELECT * 
FROM RecipientBase"""

recipient_base = pd.read_sql(recipient, con=conn)
recipient_base.head()

,mid,rno,to_eid
0,1,1,59
1,2,1,15
2,3,1,15
3,4,1,109
4,4,2,49


## 2. Data Exploration

Use the 3 dataframes to answer the following questions:

1. How many employees are there in the company?
- How many messages are there in the database?
- Convert the timestamp column in the messages. When was the oldest message sent? And the newest?
- Some messages are sent to more than one recipient. Group the messages by message_id and count the number of recepients. Then look at the distribution of recepient numbers.
    - how many messages have only one recepient?
    - how many messages have >= 5 recepients?
    - what's the highest number of recepients?
    - who sent the message with the highest number of recepients?
- plot the distribution of recepient numbers using Bokeh

In [47]:
employee_base.shape[0]

156

In [48]:
message_base['mid'].count()

21635

In [49]:
message_base['unix_time'] = pd.to_datetime(message_base['unix_time'], unit = 's')

In [60]:
print "Newest message sent:", message_base['unix_time'].max()

Newest message sent: 2002-06-21 13:37:34


In [59]:
print "Oldest message sent:", message_base['unix_time'].min()

Oldest message sent: 1998-11-13 04:07:00


In [61]:
message_base.groupby(['mid'])['mid'].count()

mid
1        1
2        1
3        1
4        1
5        1
6        1
7        1
8        1
9        1
10       1
11       1
12       1
13       1
14       1
15       1
16       1
17       1
18       1
19       1
20       1
21       1
22       1
23       1
24       1
25       1
26       1
27       1
28       1
29       1
30       1
        ..
21606    1
21607    1
21608    1
21609    1
21610    1
21611    1
21612    1
21613    1
21614    1
21615    1
21616    1
21617    1
21618    1
21619    1
21620    1
21621    1
21622    1
21623    1
21624    1
21625    1
21626    1
21627    1
21628    1
21629    1
21630    1
21631    1
21632    1
21633    1
21634    1
21635    1
Name: mid, dtype: int64

Rescale to investigate the tail of the curve

## 3. Data Merging

Use the pandas merge function to combine the information in the 3 dataframes to answer the following questions:

1. Are there more Men or Women employees?
- How is gender distributed across departments?
- Who is sending more emails? Men or Women?
- What's the average number of emails sent by each gender?
- Are there more Juniors or Seniors?
- Who is sending more emails? Juniors or Seniors?
- Which department is sending more emails? How does that relate with the number of employees in the department?
- Who are the top 3 senders of emails? (people who sent out the most emails)

Also answer the following questions regarding received messages

- Who is receiving more emails? Men or Women?
- Who is receiving more emails? Juniors or Seniors?
- Which department is receiving more emails? How does that relate with the number of employees in the department?
- Who are the top 5 receivers of emails? (people who received the most emails)

Which employees sent the most 'mass' emails?

Keep exploring the dataset, which other questions would you ask?

Work in pairs. Give each other a challenge and try to solve it.